# Práctica 1: Criptografía clásica
## UAM, 2022/2023

### Maitane Gómez González
### Ana Martínez Sabiote

## 1. Sustitución monoalfabeto

## 1.a Método afín
El siguiente programa implementa el método afín.

Llamada a la función:

afin {-C|-D} {-m |Zm|} {-a N×} {-b N+} [-i f ilein] [-o f ileout]


In [ ]:
import gmpy2
from gmpy2 import mpz
import sympy
import numpy as np

In [ ]:
"""
VERSIÓN SIN GMP 

def algoritmo_euclides(a,b):
    if a%b == 0:
        return b
    else:
        return algoritmo_euclides(b, a%b)
"""

In [ ]:
def algoritmo_euclides(a,b):
    if gmpy2.t_mod(a,b) == 0:
        return b
    else:
        return algoritmo_euclides(b, gmpy2.t_mod(a,b))

In [ ]:
mcd=algoritmo_euclides(39,150)
print(mcd)

In [ ]:
algoritmo_euclides(7,15)

In [ ]:
"""
VERSIÓN SIN GMP 

def algoritmo_euclides_extendido(a,b):

    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(b%a, a)
        u=y-(b//a)*x
        v=x
        
    return mcd, u, v
"""

In [ ]:
def algoritmo_euclides_extendido(a,b):
    """
    # Condición a>b, sino las cambiamos
    if b>a:
        aux=a
        a=b
        b=aux
    """
    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(gmpy2.c_mod(b,a), a)
        u=gmpy2.sub(y,(gmpy2.mul(gmpy2.c_div(b,a),x)))
        v=x
        
    return mcd, u, v

In [ ]:
def inverso(a,m):
    result = algoritmo_euclides_extendido(a,m)
    # Comprobamos que el mcd es 1 para que exista inverso multiplicativo
    # En consecuencia, a y m determinan una función afín inyectiva
    if result[0] == 1:
        # Entonces devolvemos el coeficiente u (que acompaña a) de la Id. de Bézout
        inv=result[1]
        return inv
    else:
        print("Error")

In [ ]:
def read_input(i):
    # Primero tomamos el input de i o de la entrada estándar
    if i==0:
        cadena=input()
    else:
        file=open(i, "r")
        cadena=file.read()
        file.close()
    print("Cadena: {}".format(cadena))
    return cadena

In [ ]:
def print_output(o,cadena):
    if o==0:
        print("Cadena: {}".format(cadena))
    else:
        file=open(o, "w")
        cadenaToStr = ' '.join([str(elem) for elem in cadena])
        file.write(cadenaToStr)
        file.close()

In [406]:
def afin(modo,m,a,b,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyz'
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            print_output(o,cadena_cifrada)
        elif modo=="-D":
            cadena_cifrada=read_input(i)
            cadena_cifrada=cadena_cifrada.split(", ")
            cadena_descifrada=[]
            cadena_texto=[]
            inv=inverso(a,m)
            for i in range(len(cadena_cifrada)):
                cadena_cifrada[i]=int(cadena_cifrada[i])
            for k in cadena_cifrada:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
                if k_descifrado<0:
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
                cadena_texto.append(alfabeto[k_descifrado])
            
            print_output(o, cadena_texto)
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [ ]:
afin("-C",26,23,3,"cadena.txt")

In [ ]:
afin("-C",130,16,27)

In [ ]:
afin("-C",52,23,3)

In [ ]:
afin("-D",52,23,3)

In [ ]:
afin("-D",52,23,3,"cadena_cifrada.txt","resultado.txt")

## 1.b Criptoanálisis del cifrado afín

In [ ]:
def phi(m):
    if m ==1:
        return 1
    else:
        if gmpy2.is_prime(m):
            return (m-1)
        else:
            for i in range(m):
                gmpy2.mul(algoritmo_euclides_extendido())

In [407]:
alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
digrama=([])
for i in alfabeto:
    for j in alfabeto:
        digrama.append(i+j)



In [408]:
def afin_no_trivial(modo, m, a, b, i=0, o=0):
    
    alfabeto='abcdefghijklmnopqrstuvwxyz'
    digrama=([]) #generamos un vetor de di-gramas del alfabeto que hemos declarado arriba
    for i in alfabeto:
        for j in alfabeto:
            digrama.append(i+j)
    
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            
            cadena=read_input(i)
            #cadena=input()
            
            #Traducimos los caracteres a números
            cadena_numerica=[]
            
            j=0
            if len(cadena)%2==0:
                while j <(len(cadena)-1):
                    cadena_numerica.append(digrama.index(cadena[j]+cadena[j+1]))
                    j=j+2
            else:
                while j <(len(cadena)-2):
                    cadena_numerica.append(digrama.index(cadena[j]+cadena[j+1]))
                    j=j+2
                cadena_numerica.append(alfabeto.index(cadena[len(cadena)-1])) #como es impar, la última letra la ciframos aparte
                
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            print_output(o,cadena_cifrada)
            
        elif modo=="-D":
            
            #cadena_cifrada=read_input(i)
            cadena_cifrada=input()
            cadena_cifrada=cadena_cifrada.split(", ")
            cadena_descifrada=[]
            cadena_texto=[]
            inv=inverso(a,m)
            
            for i in range(len(cadena_cifrada)):
                cadena_cifrada[i]=int(cadena_cifrada[i])
            
            for k in cadena_cifrada:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
              
                if k_descifrado<0:
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
          
            for k in cadena_descifrada:
                    cadena_texto.append(digrama[k])
                    
            if len(cadena_descifrada)%2!=0:
                cadena_texto[len(cadena_texto)-1]=alfabeto[cadena_descifrada[(len(cadena_descifrada)-1)]]
            
            print_output(o, cadena_texto)
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [409]:
afin_no_trivial("-C",675,23,3,0,0)

FileNotFoundError: [Errno 2] No such file or directory: 'z'

In [ ]:
afin_no_trivial("-D",675,23,3)

In [ ]:
def fortaleza(m):
    z_m_inv=sympy.totient(m) #calculamos la funcion phi
    return gmpy2.mul((m),z_m_inv) #calculamos la fortaleza multiplicando Zm y Zm*


In [ ]:
fortaleza(26**26+26)
fortaleza(26)
fortaleza(52)


In [ ]:
El cifrado afin muy vulnerable a los ataques. Principalmente al de analisis 
de frecuencias y busqueda en el espacio de claves.

In [438]:
def cambiar_orden(castellano):
    
    aux=np.array([])
    for i in range(len(castellano)):
        cambios=np.array([])
        cambios=np.append(cambios,castellano[i])
        for j in range(len(castellano)):
            if castellano[j] not in cambios:
                cambios=np.append(cambios,castellano[j])
        aux=np.append(aux,cambios)
        
    for i in range(len(castellano)):
        cambios=np.array([])
        cambios=np.append(cambios,castellano[i])            
        for j in reversed(range(len(castellano))):
            if castellano[j] not in cambios:
                cambios=np.append(cambios,castellano[j])
        aux=np.append(aux,cambios)
    
    lista=np.reshape(aux, (2*len(castellano), len(castellano)))
    
    
    return lista

In [439]:
def cambiar_ord(castellano):
    auxi=castellano
    aux=np.array([])
    
    i=0
    aux=np.append(aux,castellano)
    while i <len(castellano)-1:
        
        for j in range(len(castellano)):
            
        
        a=auxi[0]
        auxi=np.insert(auxi,len(auxi),a)
        print(auxi)
        auxi=np.delete(auxi,0)
        print(auxi)
        aux=np.append(aux,auxi)
        i=i+1
        
    lista=np.reshape(aux, (len(castellano), len(castellano)))
    print(lista)
        

IndentationError: expected an indented block (2653160292.py, line 12)

In [440]:
cambiar_ord(['a', 'b', 'c'])


['a' 'b' 'c' 'a']
['b' 'c' 'a']
['b' 'c' 'a' 'b']
['c' 'a' 'b']
[['a' 'b' 'c']
 ['b' 'c' 'a']
 ['c' 'a' 'b']]


In [441]:
cambiar_orden(['a', 'b', 'c', 'e', 'f'])

array([['a', 'b', 'c', 'e', 'f'],
       ['b', 'a', 'c', 'e', 'f'],
       ['c', 'a', 'b', 'e', 'f'],
       ['e', 'a', 'b', 'c', 'f'],
       ['f', 'a', 'b', 'c', 'e'],
       ['a', 'f', 'e', 'c', 'b'],
       ['b', 'f', 'e', 'c', 'a'],
       ['c', 'f', 'e', 'b', 'a'],
       ['e', 'f', 'c', 'b', 'a'],
       ['f', 'e', 'c', 'b', 'a']], dtype='<U32')

In [442]:
from collections import Counter

cadena = input()
text= cadena.split(", ")
letters = Counter(text)
#letras=np.array(list(letters.items()))
print("cantidad de letras = ", len(letters))
print(letters)

castellano=(['a', 11.96],['b', 0.92],['c', 2.92],['d', 6.87],['e', 16.78],['f', 0.52],['g', 0.73],
           ['h', 0.89],['i', 4.15],['j', 0.3],['k', 0.0],['l', 8.37],['m', 2.12],['n', 7.01],
           ['o', 8.69],['p', 2.77],['q', 1.53],['r', 4.94],['s', 7.88],['t', 3.31],['u', 4.80],
           ['v', 0.39],['w', 0.0],['x', 0.06],['y', 1.54],['z', 0.15])

ingles=(['a', 11.96],['b', 1.54],['c', 3.06],['d', 3.99],['e', 12.51],['f', 2.30],['g', 1.96],
        ['h', 0.89],['i', 7.26],['j', 0.16],['k', 0.67],['l', 4.14],['m', 2.53],['n', 7.09],
       ['o', 7.60],['p', 2.0],['q', 0.11],['r', 6.12],['s', 6,54],['t', 9.25],['u', 2.71],
       ['v', 0.99],['w', 1.92],['x', 1.92],['y', 1.73],['z', 0.19])

letras=sorted(letters, reverse=True)
castellano_ord=sorted(castellano, key=lambda letra: letra[1], reverse=True)
ingles_ord=sorted(ingles, key=lambda letra: letra[1], reverse=True)

aux=np.array([])
for i in range(len(castellano_ord)):
    aux=np.append(aux, castellano_ord[i][0])

pruebas=cambiar_orden(aux)
print(len(pruebas[0]))
lista=list(letters.elements())
cambio=[]

for j in range(len(pruebas)):
    texto=[]
    for elem in text:
        for i in range(len(lista)):
            if elem==lista[i]: #si su porcentaje es el i-esimo
                texto.append(pruebas[j][i]) #lo intercambiamos por el i-esimo componente de la prueba actual
                #cambio.append(castellano_ord[i][0])
                
                break
    
    
    print(texto)
    
    print("es una solucion valida S/N")
    print("otra posible solucion")
    if input()=="S":
        break
    
    
    

hola
cantidad de letras =  1
Counter({'hola': 1})
26
['e']
es una solucion valida S/N
otra posible solucion

['a']
es una solucion valida S/N
otra posible solucion

['o']
es una solucion valida S/N
otra posible solucion

['l']
es una solucion valida S/N
otra posible solucion

['s']
es una solucion valida S/N
otra posible solucion

['n']
es una solucion valida S/N
otra posible solucion

['d']
es una solucion valida S/N
otra posible solucion

['r']
es una solucion valida S/N
otra posible solucion

['u']
es una solucion valida S/N
otra posible solucion

['i']
es una solucion valida S/N
otra posible solucion

['t']
es una solucion valida S/N
otra posible solucion

['c']
es una solucion valida S/N
otra posible solucion

['p']
es una solucion valida S/N
otra posible solucion

['m']
es una solucion valida S/N
otra posible solucion

['y']
es una solucion valida S/N
otra posible solucion

['q']
es una solucion valida S/N
otra posible solucion

['b']
es una solucion valida S/N
otra posible soluc

In [443]:
afin("-C",26,23,3)


hola
Cadena: hola
Cadena: [8, 13, 22, 3]


In [ ]:
#Para los casos B,C,D y E

In [ ]:
def criptoanalisis_afin(claro1=0,cifrado1=0, claro2=0, cifrado2=0,o):
   #obtenemos la primera pareja de textos
    if claro1==0:
        claro1=input()
    else:
        file=open(i, "r")
        claro1=file.read()
        file.close()
    
    if cifrado1==0:
        cifrado1=input()
    else:
        file=open(i, "r")
        cifrado1=file.read()
        file.close()
    
    #obtenemos la segunda pareja de textos
    if claro2==0:
        claro2=input()
    else:
        file=open(i, "r")
        claro2=file.read()
        file.close()
    
    if cifrado2==0:
        cifrado2=input()
    else:
        file=open(i, "r")
        cifrado2=file.read()
        file.close()
    
    #y1=a*x1+b%m -> a*x1=y1-b%m ->a=invX1*(y1-b%m)
    #y2=a*x2+b%m->b%m= y2-a*x3
    
    

In [ ]:
#Análisis de frecuencia
El análisis de frecuencia es el estudio de la frecuencia de letras o grupos
de letras en un texto cifrado.

## 2. Sustitución polialfabeto

## 2.a Método de Hill

In [ ]:
import numpy as np
import os
import math
import copy

In [ ]:
def determinante(matriz):
   
    if len(matriz)==2 and len(matriz[0])==2:
        #calculamos el determinante
        det=matriz[0][0]*matriz[1][1]-(matriz[1][0]*matriz[0][1])
       
        return det
    else:
        suma=0
        for i in range(len(matriz)): #calculamos el determinante por cofactores
            maux=copy.deepcopy(matriz)
            maux.remove(matriz[0]) #eliminamos la primera fila
            for j in range(len(maux)):
                maux[j]=maux[j][0:i]+maux[j][i+1:]
                
         
            suma= suma+ (-1)**((i+j)%2)*matriz[0][i]*determinante(maux)
            
        return suma
        

In [ ]:
matriz = [[1,2,3], [3,4,5], [1,4,3]]
print(determinante(matriz))
matriz = [[11,8], [3,7]]
print(determinante(matriz))

In [ ]:
def adjunto(matriz):
    adjunto=np.zeros(np.shape(matriz))
    if len(matriz)==2 and len(matriz[0])==2:
         #calculamos el adjunto
        adjunto[0][0]=matriz[1][1]
        adjunto[0][1]=-matriz[0][1]
        adjunto[1][0]=-matriz[1][0]
        adjunto[1][1]=matriz[0][0]
        
        return adjunto
    else:
        
        for i in range(len(matriz)):
            maux=copy.deepcopy(matriz)
            for j in range(len(matriz)):
             
                maux=np.delete(matriz,i,0)
                aux=np.delete(maux,j,1)
                auxi=aux.tolist()
                #la matriz de cofactores transpuesta es el djunto
                adjunto[j][i]=(-1)**((i+j)%2)*determinante(auxi)
            
                
        return adjunto

In [ ]:
matriz = [[11,8], [3,7]]
print(adjunto(matriz))

In [ ]:
def inversa(matriz,modulo):
    inversa=np.zeros(np.shape(matriz))
    det=determinante(matriz)%modulo
    if det !=0:
        adj=adjunto(matriz)%modulo
        for i in range(len(matriz)):
            for j in range(len(matriz[i])):
                inversa[i][j]=(adj[i][j]/det)#%modulo
                
    return inversa%modulo #esto puede que no sea necesario porque ya estamos en matemática modular

In [ ]:
matriz = [[11,8], [3,7]]
print(inversa(matriz,26))

In [ ]:
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
np.delete(arr, 1, 0)

In [ ]:
matriz = [[1, 2, 4], [3,5,4], [1]]

# Padding a la matriz
n=3
print(matriz)
print(len(matriz[n-1]))
k=len(matriz[n-1])
if k<n:
    padding=[0]*(n)
    for i in range(0,k):
        padding[i]=matriz[n-1][i]
    matriz[n-1]=padding
  
print(matriz)
matriz=np.array(matriz)
matriz%3

In [ ]:
print(matriz[1])

In [ ]:
def cifrar(matriz_numerica, matriz,mod,n):
    matriz_cifrada=np.zeros((n,n))
    
    for i in range(len(matriz_numerica)):
        cadena_cifrada= (np.dot(matriz_numerica[i],matriz))%mod
        matriz_cifrada[i]=cadena_cifrada

    return matriz_cifrada  

In [ ]:
def descifrar(matriz_cifrada, matriz,mod,n):
    matriz_descifrada=np.zeros((n,n))
    inv=inversa(matriz,mod)
 
    for i in range(len(matriz_cifrada)):
        cadena_descifrada= (np.dot(matriz_cifrada[i],inv))%mod
        matriz_descifrada[i]=cadena_descifrada

    return matriz_descifrada  

In [ ]:
def hill(modo,mod,n,k,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyz'
  
    with open(k,'r') as f:
        datos = ''.join(f.readlines()).replace('\n',';')
    matriz = np.matrix(datos).tolist()
    f.close()
    
    # Padding a la matriz
    #ult=len(matriz[n-1])
    #if ult<n:
     #   padding=[0]*(n)
      #  for i in range(0,k):
       #     padding[i]=matriz[n-1][i]
        #matriz[n-1]=padding
    #matriz=np.array(matriz)
   
    det=np.linalg.det(matriz)
    #comprobamos que la matriz K tiene una función biyectiva
    if algoritmo_euclides(int(det),mod)==1:
       
        if modo=="-C":
           
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
                    
            # Dividimos en bloques de n elementos el texto
            # Si m no es múltiplo de n se añade padding
            m=len(cadena_numerica)/n
            max=len(cadena_numerica)
            matriz_numerica=np.zeros((math.ceil(m),n))
         
            pos=0
            for i in range(math.ceil(m)):
                for j in range(n):
                    if pos<max:
                        matriz_numerica[i][j]=cadena_numerica[pos]
                        pos=pos+1
                        
            #ciframos cadena a cadena y lo guardamos en un matriz           
            matriz_cifrada=cifrar(matriz_numerica,matriz,mod,n)
            print_output(o,matriz_cifrada)
           
            
        elif modo=="-D":
            if i==0:
                cadena_cifrada=input()
                mat_cifrada = np.matrix(cadena_cifrada).tolist()
                matriz_cifrada= np.reshape(mat_cifrada, (n,n))
               
                        
            else:
                
                with open(i,'r') as f:
                    datos = ''.join(f.readlines()).replace('\n',';')     
                f.close()
                
                matriz_cifrada = np.matrix(datos).tolist() 
        
           
            matriz_descifrada=descifrar(matriz_cifrada, matriz,mod,n)
            
            cadena_num=[]*n*n
            for i in range(len(matriz_descifrada)):
                for j in range(len(matriz_descifrada[i])):
                    
                    if matriz_descifrada[i][j]<0:
                        matriz_descifrada[i][j]=mod+matriz_descifrada[i][j]
        
                    cadena_num.append(alfabeto[int(matriz_descifrada[i][j])])
            print_output(o,cadena_num)
            
    else:
        print("{} y {} no son primos relativos. Error".format(det,mod))

In [ ]:
k = [[11,8], [3,7]]
hill("-D",26, 2,"matriz_k.txt","matriz_cifrada.txt" ,0)

In [ ]:
hill("-C",26,2, "matriz_k.txt", 0,0 )

In [ ]:
cadena_numerica=[8,15,12,1]
matriz = [[11,8], [3,7]]
n=2

m=len(cadena_numerica)/n
max=len(cadena_numerica)
matriz_numerica=np.zeros((math.ceil(m),n))

pos=0
for i in range(math.ceil(m)):

    for j in range(n):
        
        if pos<max:
           
            matriz_numerica[i][j]=cadena_numerica[pos]
            pos=pos+1
            
print("matriz_numerica")
print(matriz_numerica)   

matriz_cifrada=cifrar(matriz_numerica, matriz,26,n)
print("matriz cifrada")
print(cifrar(matriz_numerica, matriz,26,n))

print(descifrar(matriz_cifrada, matriz,26,n))
print("matriz descifrada")
matriz_descifrada=descifrar(matriz_cifrada, matriz,26,n)
alfabeto='abcdefghijklmnopqrstuvwxyz'
cadena_num=[]*n*n
for i in range(len(matriz_descifrada)):
    for j in range(len(matriz_descifrada[i])):
        if matriz_descifrada[i][j]<0:
            matriz_descifrada[i][j]=26+matriz_descifrada[i][j]
        cadena_num.append(alfabeto[int(matriz_descifrada[i][j])-1])
print(cadena_num)

In [ ]:
with open('matriz_k.txt','r') as f:
    datos = ''.join(f.readlines()).replace('\n',';')

matriz = np.matrix(datos)
print(matriz)

## 2.b Método de Vigenere

In [ ]:
def vigenere(modo,k,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    base=len(alfabeto)
    n=len(k)
    #Traducimos la clave de caracteres a números
    k_numerica=[]
    for j in k:
        if j in alfabeto: 
            k_numerica.append(alfabeto.index(j))
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Dividimos en bloques de n elementos el input
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_numerica)/n
        maxi=len(cadena_numerica)
        matriz_numerica=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_numerica[i][j]=cadena_numerica[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que cifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_numerica.shape[0]
        elementos=matriz_numerica.shape[1]
        #print("Matriz numerica")
        #print(matriz_numerica)
        matriz_cifrada=np.zeros((filas,elementos))
        #print("Matriz cifrada de ceros")
        #print(matriz_cifrada)
        if elementos==n:
            print("Bloques ok")
        for i in range(filas):
            for j in range(elementos):
                matriz_cifrada[i][j]=(matriz_numerica[i][j]+k_numerica[j])%base
        #print("Matriz cifrada")
        #print(matriz_cifrada)
        cadena_cifrada=np.concatenate(matriz_cifrada)
        #print(cadena_cifrada)
        print_output(o,cadena_cifrada)
    elif modo=="-D":
        cadena_cifrada=read_input(i)
        cadena_cifrada=cadena_cifrada.split(" ")
        #for i in range(len(cadena_cifrada)):
        #   cadena_cifrada[i]=int(cadena_cifrada[i])
        # Dividimos en bloques de n elementos el texto cifrado
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_cifrada)/n
        maxi=len(cadena_cifrada)
        matriz_cifrada=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_cifrada[i][j]=cadena_cifrada[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que descifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_cifrada.shape[0]
        elementos=matriz_cifrada.shape[1]
        matriz_descifrada=np.zeros((filas,elementos))
        if elementos==n:
            print("Bloques ok")
        for i in range(filas):
            for j in range(elementos):
                matriz_descifrada[i][j]=(matriz_cifrada[i][j]-k_numerica[j])%base
        cadena_descifrada=np.concatenate(matriz_descifrada)
        cadena_texto=[]
        for i in range(len(cadena_descifrada)):
            #cadena_cifrada[i]=int(cadena_cifrada[i])
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])
        print_output(o,cadena_texto)

In [ ]:
vigenere("-C", "clave")

In [ ]:
vigenere("-D", "clave")


In [ ]:
vigenere("-C", "clave", "texto_vigenere.txt", "resultado_vigenere.txt")

In [ ]:
vigenere("-D", "clave", "resultado_vigenere.txt", "descifrado_vigenere.txt")

## 2.c Criptoanálisis del cifrado de Vigenere

## 3. Cifrado de flujo

In [ ]:
def rec_fib(n):
    if n > 1:
        return rec_fib(n-1) + rec_fib(n-2)
    return n

In [ ]:
# Generador de secuencia aleatoria
def generador_aleatorio(m,cont):
    k=(rec_fib(m)%m)*m*cont
    return k

In [ ]:
# Ejemplo de secuencia cifrante de 5 elementos para clave 14
m=14
for i in range(5):
    k=generador_aleatorio(m,i)
    print(k)

In [ ]:
# m es la clave
# n es el tamaño de la secuencia de claves
def flujo(modo,m,n,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Ciframos carácter a carácter
        cadena_cifrada=[]
        count=0
        for i in cadena_numerica:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_cifrada.append(int(bin(i^k)[2:]))
            count=count+1
        print_output(o,cadena_cifrada)
    elif modo=="-D":
        cadena_cifrada=read_input(i)
        cadena_cifrada=cadena_cifrada.split(" ")
        cadena_descifrada=[]
        cadena_texto=[]
        for i in range(len(cadena_cifrada)):
            cadena_cifrada[i]=int(cadena_cifrada[i],2)
        count=0
        for i in cadena_cifrada:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_descifrada.append((i^k))
            count=count+1
        for i in range(len(cadena_descifrada)):
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])
        print_output(o,cadena_texto)

In [ ]:
flujo("-C", 4,2)

In [ ]:
flujo("-D", 4,2)

In [ ]:
flujo("-C", 10,20)

In [ ]:
flujo("-D", 10, 20)